This notebook is part of the *orix* documentation https://orix.rtfd.io. Links to the documentation won’t work from the notebook.

# Crystal maps

This notebook details how to load and save crystallographic mapping data in
orix, as well as analysing and visualising the data. All interactions with this
type of data is done with the
[orix.crystal_map.CrystalMap](reference.rst#orix.crystal_map.CrystalMap) class.

Orientations and other properties acquired from a super-duplex stainless steel
EBSD data set with two phases, austenite and ferrite, are used as example data.
The data is available here: http://folk.ntnu.no/hakonwii/files/orix-demos/,
courtesy of Prof. Jarle Hjelen (Norwegian University of Science and Technology,
Norway).

In [ ]:
%matplotlib qt5

from diffpy.structure import Atom, Lattice, Structure
import numpy as np

from orix import crystal_map, io, quaternion

# Load, create and save

A CrystalMap instance can be obtained by reading an orientation data set stored
in a format supported by orix using the load function, or by passing the
necessary arrays to the `CrystalMap.__init__()` method. Two file formats are
supported, in addition to orix's own HDF5 format: Data in the .ang format
produced by the softwares EDAX TSL OIM Data Collection v7, NanoMegas ASTAR
Index, and EMsoft v4/v5 via the `EMdpmerge` program, and data in EMsoft v4/v5
HDF5 files produced by the `EMEBSDDI` program.

Two writers are supported, namely orix's own HDF5 format, readable by orix only,
and the .ang format, readable at least by MTEX and EDAX TSL OIM Analysis v7.

## Load or create

Let's load a crystal map from an .ang file produced by EMsoft and inspect it

In [ ]:
datadir = "/home/hakon/phd/data/jarle_emsoft/sdss/em/"
fname = "sdss_ferrite_austenite.ang"

xmap = io.load(datadir + fname)
print(xmap)
xmap.plot(overlay="dp")  # Dot product values added to the alpha (RGBA) channel

The indexing properties returned by EMsoft in their .ang files are the pattern
image quality (iq) (according to Niels Krieger Lassen's method), and the highest
normalized dot product (dp) between the experimental and best matching simulated
pattern.

The same `CrystalMap` object can be obtained by reading each array from the .ang
file ourselves and passing this to `CrystalMap.__init__()`

In [ ]:
# Read each column from the file
euler1, euler2, euler3, x, y, iq, dp, phase_id = np.loadtxt(
    datadir + fname, unpack=True
)

# Create a Rotation object from Euler angles
euler_angles = np.column_stack((euler1, euler2, euler3))
rotations = quaternion.Rotation.from_euler(euler_angles)

# Create a property dictionary
properties = dict(iq=iq, dp=dp)

# Create unit cells of the phases
structures = [
    Structure(
        title="austenite",
        atoms=[Atom("fe", [0] * 3)],
        lattice=Lattice(0.360, 0.360, 0.360, 90, 90, 90)
    ),
    Structure(
        title="ferrite",
        atoms=[Atom("fe", [0] * 3)],
        lattice=Lattice(0.287, 0.287, 0.287, 90, 90, 90)
    ),
]
phase_list = crystal_map.PhaseList(
    names=["austenite", "ferrite"],
    point_groups=["432", "432"],
    structures=structures,
)

# Create a CrystalMap instance
xmap2 = crystal_map.CrystalMap(
    rotations=rotations,
    phase_id=phase_id,
    x=x,
    y=y,
    phase_list=phase_list,
    prop=properties,
)
xmap2.scan_unit = "um"

xmap2

## Save

### orix HDF5 format

As mentioned, the two writers implemented are orix's own HDF5 format and the
.ang format

In [ ]:
io.save(
    filename=datadir + "sdss_ferrite_austenite2.h5",
    object2write=xmap,
    overwrite=True,  # Default is False
)

Read the file contents back into a `CrystalMap` object using
[orix.io.load()](reference.rst#orix._io.load) function.

All contents in this file can be inspected using any HDF5 viewer and read back
into Python using the h5py library (which we use).

### .ang format

The .ang writer supports many use cases. Some of these are demonstrated here,
by reloading the saved crystal maps.

First, let's write the multi phase map to an .ang file, specifying that the
`xmap.dp` property should be written to the confidence index (CI) column

In [ ]:
fname_ang1 = "sdss_dp_ci.ang"
io.save(
    filename=datadir + fname_ang1,
    object2write=xmap,
    confidence_index_prop="dp"
)

xmap_reload1 = io.load(datadir + fname_ang1)
print(xmap_reload1)
print(xmap_reload1.prop)

Note that points not in data are set to `not_indexed` when reloaded from the
.ang file, and that all properties in points not in the data set are set to
zero, except for the CI column where this property in points not in the data
(the austenite points) are set to -1, which MTEX and EDAX TSL expects in these
points.

Finally, it is worth mentioning that if a map has more than one rotation/match
and phase ID per point, the index parameter can be passed to write any "layer"
of the data to file.

## Modify crystal phases

The phases are stored in a `PhaseList` object in the `CrystalMap.phases` attribute

In [ ]:
xmap.phases

### Symmetry

The point group symmetry are stored in the vendor and EMsoft files, however they
provide no space group symmetry. We can set this *per phase* by providing a
space group number (1-230) according to the International Tables of
Crystallography (useful link: http://img.chem.ucl.ac.uk/sgp/large/sgp.htm)

In [ ]:
xmap.phases[1].space_group = 225
xmap.phases[2].space_group = 229

xmap.phases

Note that this also changed the point group, because this is always determined
from the space group. But the proper point group, without any inversion or
mirror planes, stayed the same. The `space_group` attribute stores a
`diffpy.structure.spacegroups.SpaceGroup` object (https://www.diffpy.org/diffpy.structure/mod_spacegroup.html#diffpy.structure.spacegroupmod.SpaceGroup).

We can get the point group which a space group is the subgroup of

In [ ]:
print(get_point_group(200).name, get_point_group(230).name)

The point group stores symmetry operations as quaternions. We can get them as
orientation matrices

In [ ]:
xmap.phases[1].point_group[:2]

In [ ]:
xmap.phases[1].point_group[:2].to_matrix()

`diffpy.structure` stores rotation symmetry operations as orientation matrices
and translations as 1D arrays

In [ ]:
[(i.R, i.t) for i in xmap.phases[1].space_group.symop_list[:2]]

We can get the quaternion representation of these matrices

In [ ]:
[Rotation.from_matrix(i.R) for i in xmap.phases[1].space_group.symop_list[:2]]

### Index phase list

The phase list can be indexed by phase ID or name

In [ ]:
xmap.phases[1]

In [ ]:
xmap.phases["austenite"]

In [ ]:
xmap.phases[1:]

In [ ]:
xmap.phases["austenite", "ferrite"]

When asking for a single phase, either by an integer or a single string, a
`Phase` object was returned. In the other cases, a `PhaseList` object was
returned

In [ ]:
print(type(xmap.phases[1]), type(xmap.phases[1:]))

In [ ]:
Valid point group names to use when setting the point group symmetry are

In [ ]:
from orix.quaternion.symmetry import _groups

[point_group.name for point_group in _groups]

In [ ]:
# NBVAL_IGNORE_OUTPUT
xmap.phases["Austenite"].point_group = "-43m"

xmap.phases

Note that the `space_group` was set to `None` since space group Fm-3m is not a
subgroup of -43m.

Let's revert to the correct space group (and the name, for convenience)

In [ ]:
xmap.phases["Austenite"].name = "austenite"
xmap.phases["austenite"].space_group = 225

xmap.phases

We can add a phase by giving its name and point group symmetry

In [ ]:
xmap.phases.add(Phase("sigma", point_group="4/mmm"))

xmap.phases

When adding a phase to the phase list like this, the phases' structure contains no atoms and the default lattice parameters are used

In [ ]:
xmap.phases["sigma"].structure.lattice.abcABG()

So let's set this

In [ ]:
xmap.phases["sigma"].structure.lattice = Lattice(0.880, 0.880, 0.880, 90, 90, 90)
print(xmap.phases["sigma"].structure.lattice)

If some data points are considered as not indexed, a "not_indexed" phase can be
added to the phase list to keep track of these points

In [ ]:
xmap.phases.add_not_indexed()

xmap.phases

No points in this data set are considered not indexed. A phase list with only
the phases in the data is stored in the `phases_in_data` attribute

In [ ]:
xmap.phases_in_data

We can of course remove a phase from the phase list, either by its name or phase ID

In [ ]:
del xmap.phases["sigma"]
del xmap.phases[-1]

xmap.phases

### Properties

The phase name, space group, point group, proper point group, color and
structure can be accessed for the full phase list or a single phase

In [ ]:
print(xmap.phases.names)
print([i.short_name for i in xmap.phases.space_groups])
print([i.name for i in xmap.phases.point_groups])
print([i.proper_subgroup.name for i in xmap.phases.point_groups])
print(xmap.phases.colors)
print(xmap.phases.structures)

Note that the structures' representations are empty lists since no atoms have been added to them yet.

In [ ]:
xmap.phases["austenite"]
print(xmap.phases["austenite"].name)
print(xmap.phases["austenite"].space_group.short_name)
print(xmap.phases["austenite"].point_group.name)
print(xmap.phases["austenite"].point_group.proper_subgroup.name)
print(xmap.phases["austenite"].color)
print(xmap.phases["austenite"].structure)

These attributes (not the phase ID) can be set *per phase*

In [ ]:
xmap.phases["austenite"].name = "Austenite"

xmap.phases["Austenite"].structure = Structure(
    lattice=Lattice(0.36, 0.36, 0.36, 90, 90, 90)
)
print(xmap.phases["Austenite"].structure)

xmap.phases["Austenite"].color = "lime"  # Sets RGB tuple (0, 1, 0)
print(xmap.phases["Austenite"].color_rgb)

xmap.phases

Valid color strings can be found here: https://matplotlib.org/3.1.0/tutorials/colors/colors.html

### Create phase list

We can create a phase list by calling `PhaseList.__init__()`

In [ ]:
PhaseList(
    names=['al', 'cu'],
    space_groups=[225, 225],
    colors=['lime', 'xkcd:violet'],
    ids=[0, 1],
    structures=[
        Structure(
            atoms=[Atom("al", [0] * 3)],
            lattice=Lattice(0.405, 0.405, 0.405, 90, 90, 90)
        ),
        Structure(
            atoms=[Atom("cu", [0] * 3)],
            lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90)
        )
    ]
)

or by creating `Phase` objects and passing these to the first argument in
`PhaseList.__init__()` as a list (or single `Phase` objects)

In [ ]:
al = Phase(name='al', space_group=225, color="C0")
cu = Phase(
    color="C1",
    structure=Structure(
        title="cu",
        lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90)
    )
)

PhaseList([al, cu])

Note that the Cu phase name was retrieved from the `Structure` object.

### Copying

If we want a shallow copy of the phase list

In [ ]:
pl = xmap.phases
pl["ferrite"].color = "red"

xmap.phases

If we want a deep copy of the phase list

In [ ]:
pl = xmap.phases.deepcopy()
pl.add(Phase("chi", point_group="-43m"))
print(pl, "\n")

print(xmap.phases)

## Inspect orientation data

Orientations are stored as rotations in a `Rotation` object

In [ ]:
xmap.rotations

Orientations *per phase* can be obtained by applying the phase point group
symmetry

In [ ]:
o_austenite = xmap["austenite"].orientations

o_austenite

The above is equivalent to

In [ ]:
Orientation(xmap["austenite"].rotations).set_symmetry(
    xmap["austenite"].phases[1].point_group
)

Orientation angles and axes are readily available

In [ ]:
o_austenite.angle

In [ ]:
# Obtain as a numpy.ndarray
o_austenite.angle.data

In [ ]:
o_austenite.axis.data

## Inspect, add and delete map properties

Map properties are stored in the `CrystalMap.prop` attribute dictionary

In [ ]:
xmap.prop

All properties in this dictionary are also available directly from the `CrystalMap` as attributes

In [ ]:
xmap.iq

In [ ]:
xmap.dp

We can add a map property by specifying its name and an initial value in each map point

In [ ]:
xmap.prop["grain_boundary"] = 0

xmap.grain_boundary

In [ ]:
xmap.prop["grain_boundary2"] = np.arange(xmap.size, dtype=int)

xmap.grain_boundary2

We can also delete a property from the `prop` dictionary

In [ ]:
del xmap.prop["grain_boundary2"]

xmap.prop

## Select and modify data based upon criteria

We can select data in a crystal map in three ways:
1. by phase name or "indexed"/"not_indexed"
2. by a slice
3. by a boolean array

Getting all data belonging to one phase

In [ ]:
xmap["austenite"]

or two phases

In [ ]:
xmap["austenite", "ferrite"]

or all indexed points

In [ ]:
xmap["indexed"]

or all non-indexed points

In [ ]:
xmap["not_indexed"]

When slicing a crystal map, it is important to know the data size and shape

In [ ]:
xmap.size

In [ ]:
xmap.shape

So, to get the data within a rectangle

In [ ]:
xmap[20:50, 40:90]

The most powerful way to select data is by requiring a certain criteria

In [ ]:
dp_mean = xmap.dp.mean()
print(dp_mean)

xmap_high_dp = xmap[xmap.dp > dp_mean]
print(xmap_high_dp.dp.min())

Note that when selecting a subset of the data, a shallow copy (view) of the
crystal map is obtained. This means that whatever changes made to `xmap_high_dp`
also change `xmap`. When we want a deep copy, we use the `CrystalMap.deepcopy()`
method

In [ ]:
xmap_nobody_owns_me = xmap[xmap.dp > dp_mean].deepcopy()

We can chain the criteria

In [ ]:
xmap[(xmap.dp > 0.81) & (xmap.phase_id == 1)]

## Plotting

All map plotting is done via a `matplotlib` "projection" named "plot_map". To plot a phase map

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
im = ax.plot_map(xmap)

Hover over figure points to display the (x,y) position and orientations in that point!

Note that `plot_map()` wraps `matplotlib.axes.Axes.imshow`. All key word arguments in `plot_map()` are passed to `imshow()`, so be sure to check [its documentation](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html?highlight=imshow#matplotlib.axes.Axes.imshow) out for any additional arguments.

We can add any overlay, from any property with a value in each map point, to the map

In [ ]:
ax.add_overlay(xmap, xmap.dp)

To save our phase map with the scalebar and legend, but without white padding

In [ ]:
ax.remove_padding()
fig.savefig(
    datadir + 'phase_map.png',
    bbox_inches="tight",
    pad_inches=0,
)

To save phase map without scalebar, legend and white padding, and one image pixel per map point

In [ ]:
plt.imsave(
    datadir + 'phase_no_fluff.png',
    arr=im.get_array()  # 2D NumPy array, possibly with an RGB tuple in each element
)

We can plot any property with a value in each map point

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
im = ax.plot_map(xmap, xmap.dp, cmap="inferno")

And change the colormap later if we want to

In [ ]:
im.set_cmap("viridis")

And add a colorbar if we want

In [ ]:
cbar = ax.add_colorbar(label="Dottproduct")

Which we can update if we mispelled the label or want other adjustements

In [ ]:
cbar.ax.set_ylabel("Dot product", rotation=270);

We can also plot orientation related values, like axis and angles etc., and restrict the color bar maximum

In [ ]:
# Get rotation angles in degrees
angles = xmap.rotations.angle.data * 180 / np.pi

fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
im = ax.plot_map(xmap, angles, vmax=angles.max() - 10)

ax.add_overlay(xmap, xmap.iq)

ax.add_colorbar(label="Rotation angle");

To plot only one phase, while passing custom
* scalebar properties (https://matplotlib.org/mpl_toolkits/axes_grid/api/anchored_artists_api.html#mpl_toolkits.axes_grid1.anchored_artists.AnchoredSizeBar)
* legend properties (https://matplotlib.org/3.3.0/api/_as_gen/matplotlib.pyplot.legend.html)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
im = ax.plot_map(
    xmap["austenite"],
    scalebar=True,  # False for removed
    scalebar_properties={
        "loc": 4,  # 1: upper right, 2: upper left, etc. counter-clockwise
        "frameon": False,
        "sep": 6,  # Vertical spacing between bar and text
        "size_vertical": 0.2,  # Bar height
    },
    legend_properties={
        "framealpha": 1,  # 0: fully transparent, 1: opaque
        "handlelength": 1.5,  # Colored square width
        "handletextpad": 0.1,  # Horizontal space between square and text
        "borderpad": 0.1,
    },
)

Plot only a rectangle of the map

In [ ]:
xmap2 = xmap[20:50, 40:90]

fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
ax.plot_map(xmap2)
ax.add_overlay(xmap2, xmap2.dp)

Plot only parts of a map based on chained conditionals, like belonging to one phase or having a property value above a threshold

In [ ]:
# Conditional slicing
xmap2 = xmap[xmap.dp > 0.81]

fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
ax.plot_map(xmap2, xmap2.iq, cmap="gray")
ax.add_colorbar("Image quality")

# Chained conditional slicing
xmap2 = xmap[(xmap.dp > 0.81) & (xmap.phase_id == 1)]

fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
ax.plot_map(xmap2, xmap2.dp, cmap="cividis")
ax.add_colorbar("Dot product");

Plot histogram of a property per phase

In [ ]:
# Property of interest
this_prop = 'dp'

# Plot phase map again to see color changes
fig, ax = plt.subplots(subplot_kw=dict(projection="plot_map"))
ax.plot_map(xmap)

# Add overlay, passing str (can also pass numpy.ndarray)
ax.add_overlay(xmap, this_prop)

# Remove figure padding
ax.remove_padding()

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in xmap.phases_in_data:
    labels.append(p.name)
    colors.append(p.color)

    # Accessing the property dictionary directly
    data.append(xmap[p.name].prop[this_prop])
    # or indirectly
    #data.append(xmap[p.name].dp)

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data,
    bins=20,
    histtype='bar',
    density=True,
    label=labels,
    color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();